In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt

# Percorso al file di log di TensorBoard
log_file = 'logs/train/events.out.tfevents.1731834125.DESKTOP-D4HA6VL.3990033.2.v2'

# Inizializza le liste per memorizzare i valori di accuracy e loss
epochs = []
accuracy = []
loss = []

# Itera sugli eventi nel file di log
for event in tf.compat.v1.train.summary_iterator(log_file):
    for value in event.summary.value:
        if value.tag == 'epoch_accuracy':
            epochs.append(event.step)
            accuracy.append(value.simple_value)
        elif value.tag == 'epoch_loss':
            loss.append(value.simple_value)

# Crea i grafici per accuracy e loss
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.plot(epochs, accuracy, label='Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Training Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(epochs, loss, label='Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training Loss')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
%load_ext tensorboard


In [ ]:
%tensorboard --logdir logs

In [ ]:
%tensorboard --logdir logs_FT

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Impostazioni per migliorare l'estetica dei grafici
sns.set_theme(style="whitegrid")
plt.rcParams["font.size"] = 12
plt.rcParams["figure.figsize"] = (12, 8)
plt.rcParams["axes.titlesize"] = 16
plt.rcParams["axes.labelsize"] = 14

# Funzione per creare un singolo grafico
def plot_metric(data, title, xlabel, ylabel, labels, save_as=None):
    plt.figure()
    for df, label in zip(data, labels):
        sns.lineplot(data=df, x="Step", y="Value", label=label, linewidth=2.5)
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.legend(title="Dataset")
    plt.tight_layout()
    if save_as:
        plt.savefig(save_as, dpi=300)
    plt.show()

# Funzione per adattare le epoche del fine-tuning
def adjust_fine_tuning_epochs(dataframes, max_standard_epoch=24):
    adjusted_dataframes = []
    for df in dataframes:
        df_adjusted = df.copy()
        df_adjusted["Step"] += max_standard_epoch  # Aggiusta le epoche
        adjusted_dataframes.append(df_adjusted)
    return adjusted_dataframes

# Caricamento dei file CSV (aggiungere i tuoi file qui)
csv_files = {
    "accuracy": [
        ("train.csv", "Training Accuracy"),
        ("validation.csv", "Validation Accuracy"),
        ("train1.csv", "Fine-tuned Training Accuracy"),
        ("validation1.csv", "Fine-tuned Validation Accuracy"),
    ],
    "top5_accuracy": [
        ("top.csv", "Training Top-5 Accuracy"),
        ("topv.csv", "Validation Top-5 Accuracy"),
        ("topft.csv", "Fine-tuned Training Top-5 Accuracy"),
        ("topvft.csv", "Fine-tuned Validation Top-5 Accuracy"),
    ],
    "learning_rate": [
        ("lr.csv", "Learning Rate"),
        ("lrft.csv", "Fine-tuned Learning Rate"),
    ],
    "loss": [
        ("loss.csv", "Training Loss"),
        ("lossv.csv", "Validation Loss"),
        ("lossft.csv", "Fine-tuned Training Loss"),
        ("lossvft.csv", "Fine-tuned Validation Loss"),
    ],
    "eval_accuracy": [
        ("evalit.csv", "Validation Accuracy"),
        ("evalift.csv", "Fine-tuned Validation Accuracy"),
    ],
    "eval_loss": [
        ("eval_loss_vs_iter.csv", "Validation Loss"),
        ("fine_tuned_eval_loss_vs_iter.csv", "Fine-tuned Validation Loss"),
    ],
}

# Funzione per adattare le iterazioni del fine-tuning
def adjust_fine_tuning_iterations(dataframes, max_standard_iteration):
    adjusted_dataframes = []
    for df in dataframes:
        df_adjusted = df.copy()
        df_adjusted["Step"] += max_standard_iteration  # Aggiusta le iterazioni
        adjusted_dataframes.append(df_adjusted)
    return adjusted_dataframes

# Loop per creare i grafici
for metric, files in csv_files.items():
    data = []
    labels = []
    fine_tuning_data = []
    fine_tuning_labels = []
    
    for file, label in files:
        try:
            df = pd.read_csv(file)
            if "Fine-tuned" in label:
                fine_tuning_data.append(df)  # Salva dati fine-tuning
                fine_tuning_labels.append(label)
            else:
                data.append(df)  # Salva dati standard
                labels.append(label)
        except FileNotFoundError:
            print(f"File {file} non trovato. Saltato.")
    
    # Adattamento delle epoche o iterazioni per il fine-tuning
    if fine_tuning_data:
        if metric in ["eval_accuracy", "eval_loss"]:
            # Adatta le iterazioni per i grafici vs Iteration
            max_standard_iteration = max(df["Step"].max() for df in data) if data else 0
            fine_tuning_data = adjust_fine_tuning_iterations(fine_tuning_data, max_standard_iteration)
        else:
            # Adatta le epoche per gli altri grafici
            fine_tuning_data = adjust_fine_tuning_epochs(fine_tuning_data, max_standard_epoch=24)
    
    # Aggiungi i dati del fine-tuning ai dati principali
    data.extend(fine_tuning_data)
    labels.extend(fine_tuning_labels)
    
    # Titoli specifici per metrica
    if metric == "accuracy":
        title = "Accuracy (Training, Validation, Fine-tuned)"
        ylabel = "Accuracy"
    elif metric == "top5_accuracy":
        title = "Top-5 Accuracy (Training, Validation, Fine-tuned)"
        ylabel = "Top-5 Accuracy"
    elif metric == "learning_rate":
        title = "Learning Rate During Training"
        ylabel = "Learning Rate"
    elif metric == "loss":
        title = "Loss (Training, Validation, Fine-tuned)"
        ylabel = "Loss"
    elif metric == "eval_accuracy":
        title = "Evaluation Accuracy vs Iteration (Including Fine-tuned)"
        ylabel = "Accuracy"
    elif metric == "eval_loss":
        title = "Evaluation Loss vs Iteration (Including Fine-tuned)"
        ylabel = "Loss"
    else:
        title = metric.capitalize()
        ylabel = metric.capitalize()
    
    # Disegna il grafico se ci sono dati validi
    if data:
        xlabel = "Epoch" if metric not in ["eval_accuracy", "eval_loss"] else "Iteration"
        plot_metric(data, title, xlabel, ylabel, labels, save_as=f"{metric}.png")


In [ ]:
# Load CSV files for Learning Rate
data_lr_train = pd.read_csv("train.csv")
data_lr_finetune_train = pd.read_csv("train1.csv")
plot_combined(data_train, data_val, data_finetune_train, data_finetune_val, parameter, ylabel, title)

In [ ]:
# Load CSV files for Learning Rate
data_lr_train = pd.read_csv("learning_rate_training.csv")
data_lr_finetune_train = pd.read_csv("learning_rate_finetune_training.csv")

# Plot Learning Rate
plot_learning_rate(data_lr_train, data_lr_finetune_train)